In [33]:
import os, re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import sklearn.metrics
from scipy.linalg import orthogonal_procrustes
from itertools import permutations, combinations
dur = 40
emb_dim = 3
N_angles = 8

# directory = './data_NER/M1 4+6+6/NER_Chewie2016/NPZ/'
# directory = './data_NER/M1 4+6+6/NER_Chewie2015/NPZ/' 

# directory = './data_NER/M1 4+6+6/Cebra_16D_Chewie2016/NPZ/'
# directory = './data_NER/M1 4+6+6/Cebra_16D_Chewie2015/NPZ/'

# directory = './data_NER/M1 4+6+6/piVAE_Chewie2016/NPZ/'
# directory = './data_NER/M1 4+6+6/piVAE_Chewie2015/NPZ/'

# directory = './data_NER/M1 4+6+6/UMAP_Chewie2016/NPZ/'
# directory = './data_NER/M1 4+6+6/UMAP_Chewie2015/NPZ/'

# directory = './data_NER/M1 4+6+6/UMAP_label_Chewie2016/NPZ/'
# directory = './data_NER/M1 4+6+6/UMAP_label_Chewie2015/NPZ/'

# directory = './data_NER/PMd 6+6/NER_Chewie2016/NPZ/' 
# directory = './data_NER/PMd 6+6/Cebra_Chewie2016/NPZ/' 
# directory = './data_NER/PMd 6+6/piVAE_Chewie2016/NPZ/' 

# directory = './data_NER/S1 3/NER_Han2017/NPZ/'
# directory = './data_NER/S1 3/Cebra_Han2017/NPZ/'
# directory = './data_NER/S1 3/piVAE_Han2017/NPZ/'

name_range = slice(-28, -22)

if "piVAE" in directory:
    name_range = slice(-19, -13)
if "UMAP" in directory:
    name_range = slice(-19, -13)
if "PMd" in directory:
    name_range = slice(-29, -23)
if "S1" in directory:    
    dur = 35
if "16D" in directory:    
    emb_dim = 16
file_save = directory[:-5]+'.npz'

def self_var_decode(file_path1):
    Monkey_A = np.load(file_path1)
    XYTarget_A = np.concatenate((Monkey_A['continuous_index_train'], Monkey_A['continuous_index_test']), axis=0)
    emb_A = np.concatenate((Monkey_A['cebra_veldir_train'], Monkey_A['cebra_veldir_test']), axis=0)
    if np.max(XYTarget_A[:, 2])>10: ### angles in 0-45-90-...315degrees
        XYTarget_A[:, 2] = XYTarget_A[:, 2]/45

    train_trial_A = int(Monkey_A['continuous_index_train'].shape[0]/dur)
    test_trial_A = int(Monkey_A['continuous_index_test'].shape[0]/dur)

    XYTarget_A_train = XYTarget_A[:train_trial_A*dur, :]
    emb_A_train = emb_A[:train_trial_A*dur, :]
    XYTarget_A_test = XYTarget_A[-test_trial_A*dur:, :]
    emb_A_test = emb_A[-test_trial_A*dur:, :]

    velocity_reshaped = XYTarget_A_train[:, 0:2].reshape(train_trial_A, dur, 2)
    locations = np.cumsum(velocity_reshaped, axis=1)
    truth_XY = locations.reshape(train_trial_A*dur, 2)
    
    X = emb_A_train
    y = XYTarget_A_train[:, 0:2]
    knr = GridSearchCV(KNeighborsRegressor(), {'n_neighbors': [2,4,8,16,32,64,128,256,512,1024]}, n_jobs=8)
    knr.fit(X, y) 
    pred_vel = knr.predict(emb_A_train)
    y_C = XYTarget_A_train[:, 2]
    knc = GridSearchCV(KNeighborsClassifier(), {'n_neighbors': [2,4,8,16,32,64,128,256,512,1024]}, n_jobs=8)
    knc.fit(X, y_C)
    pred_dir_train = knc.predict(X)
    
    velocity_reshaped = pred_vel.reshape(train_trial_A, dur, 2)
    locations = np.cumsum(velocity_reshaped, axis=1)
    pred_XY = locations.reshape(train_trial_A*dur, 2)
    
    posi_train_r2 = sklearn.metrics.r2_score(truth_XY, pred_XY)
    vel_train_r2 = sklearn.metrics.r2_score(XYTarget_A_train[:, 0:2], pred_vel) ### == reg.score(X, y)
    
    target_angle_train = XYTarget_A_train[:, 2]
    pred_dir_acc = np.zeros((dur, train_trial_A))
    for i in range(train_trial_A):
        t_pred = pred_dir_train[dur*(i):dur*(i+1)]
        t_truth = target_angle_train[dur*(i):dur*(i+1)]
        pred_dir_acc[np.where(t_pred == t_truth), i] = 1
    acc_time_train = 100*np.sum(pred_dir_acc, axis=1)/train_trial_A
    pred_max_acc_train = np.max(acc_time_train)
    
    ###******** this part will use previous trained "knr & knc" ###********
    ###******** this part will use previous trained "knr & knc" ###********
    pred_vel_test = knr.predict(emb_A_test)
    velocity_reshaped = XYTarget_A_test[:, 0:2].reshape(test_trial_A, dur, 2)
    locations = np.cumsum(velocity_reshaped, axis=1)
    truth_XY = locations.reshape(test_trial_A*dur, 2)
    velocity_reshaped = pred_vel_test.reshape(test_trial_A, dur, 2)
    locations = np.cumsum(velocity_reshaped, axis=1)
    pred_XY = locations.reshape(test_trial_A*dur, 2)

    posi_test_r2 = sklearn.metrics.r2_score(truth_XY, pred_XY)
    vel_test_r2 = sklearn.metrics.r2_score(pred_vel_test, XYTarget_A_test[:, 0:2])

    pred_dir_test = knc.predict(emb_A_test)
    
    target_angle_test = XYTarget_A_test[:, 2]
    pred_dir_acc = np.zeros((dur, test_trial_A))
    for i in range(test_trial_A):
        t_pred = pred_dir_test[dur*(i):dur*(i+1)]
        t_truth = target_angle_test[dur*(i):dur*(i+1)]
        pred_dir_acc[np.where(t_pred == t_truth), i] = 1
    acc_time_test = 100*np.sum(pred_dir_acc, axis=1)/test_trial_A
    pred_max_acc_test = np.max(acc_time_test)

    return posi_train_r2, vel_train_r2, pred_max_acc_train, acc_time_train,\
            posi_test_r2, vel_test_r2, pred_max_acc_test, acc_time_test
            
files = [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
n = len(files)
pos_R_2D = np.zeros((n, 2))
vel_R_2D = np.zeros((n, 2))
peak_acc_2D = np.zeros((n, 2))
acc_time_2D = np.zeros((n, 2, dur))
date_subjects = []
n_compare = 0

def list_and_sort_files(directory):
    # List all files
    files = [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    def extract_date(filename):
        # This regex matches 6 consecutive digits that likely represent a date in YYMMDD format
        match = re.search(r'(\d{6})', os.path.basename(filename))
        date = match.group(0) if match else '000000'  # Default to '000000' if no date is found
        # Standardize to YYMMDD if necessary
        year_prefix = '20'  # assuming all dates are after the year 2000
        return int(year_prefix + date) if len(date) == 6 else int(date)
    sorted_files = sorted(files, key=extract_date)
    return sorted_files
sorted_files=list_and_sort_files(directory)

for i, file in enumerate(sorted_files):
    print("Reading file:", file)
    posi_train_r2, vel_train_r2, pred_max_acc_train, acc_time_train, posi_test_r2, vel_test_r2, \
                pred_max_acc_test, acc_time_test = self_var_decode(file)
    pos_R_2D[i, :] = posi_train_r2, posi_test_r2
    vel_R_2D[i, :] = vel_train_r2, vel_test_r2
    peak_acc_2D[i, :] = pred_max_acc_train, pred_max_acc_test
    acc_time_2D[i, :, :] = acc_time_train, acc_time_test
    if "M1PMd" in directory:
        date = file1[-29:-23]
        suffix = file1[-7:-5]
        date_subjects.append(f"{date}{suffix}")  
    elif "M1PMd" not in directory:
        print(file[name_range])
        date_subjects.append(file[name_range])
    n_compare = n_compare+1
print('label of date:', np.unique(date_subjects))

np.savez(file_save, date_subjects = date_subjects,
         pos_R_2D=pos_R_2D, vel_R_2D=vel_R_2D, peak_acc_2D=peak_acc_2D, acc_time_2D=acc_time_2D)

Reading file: ./data/M1 4+6+6/UMAP_label_Mihili2014/NPZ/Mihili_140217_embed_M1.npz
140217
Reading file: ./data/M1 4+6+6/UMAP_label_Mihili2014/NPZ/Mihili_140218_embed_M1.npz
140218
Reading file: ./data/M1 4+6+6/UMAP_label_Mihili2014/NPZ/Mihili_140303_embed_M1.npz
140303
Reading file: ./data/M1 4+6+6/UMAP_label_Mihili2014/NPZ/Mihili_140304_embed_M1.npz
140304
Reading file: ./data/M1 4+6+6/UMAP_label_Mihili2014/NPZ/Mihili_140306_embed_M1.npz
140306
Reading file: ./data/M1 4+6+6/UMAP_label_Mihili2014/NPZ/Mihili_140307_embed_M1.npz
140307
label of date: ['140217' '140218' '140303' '140304' '140306' '140307']
